# Export nodes and links - using QGIS

This file exports the links and nodes to QGIS, such that you can perform some processing on it to obtain the links inside a zone or cordon to toll. 

Notice that the nodes (centroids) and links (connectors) related to loading the demand onto the network, are not required in QGIS. Nevertheless, they HAVE TO BE added to the network first, as they change the IDs of our links and nodes! In every STA assignment that you will do, the first C node IDs are always reserved for centroids, with C the number of centroids. Also the first C link IDs are reserved for source connectors. Sunk connectors are also present, but it seems that they do not have a fixed order in the list of IDs. 

1. At first, you import all the necessary packages and files. 

In [ ]:
import os
import geopandas as gpd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore') # hide warnings
sys.path.append("..")
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids
from dyntapy.visualization import show_network
from pyproj import Proj, transform
from shapely.geometry import Point, LineString

2. Get the correct file pathes for all the elements you will need. As explained in the introduction, this will be QGIS data map, the zoning shapefile and both the link and node shapefile.

In [ ]:
QGIS_path = os.path.dirname(os.path.realpath("__file__")) + os.path.sep + 'data_map' + os.path.sep + 'QGIS'
zoning_path = QGIS_path + os.path.sep + 'BRUSSEL_40_10_aggr_comb.shp'
node_shapefile_path = QGIS_path + os.path.sep + "nodes_BRUSSEL_40.shp"
link_shapefile_path = QGIS_path + os.path.sep + "links_BRUSSEL_40.shp"

3. The next steps are always the same:

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically

  One of the important steps that you should not forget is projecting everything in the correct coordinate reference system (CRS). Especially when zones are aggregated using QGIS, this is something that you should be aware of. 

  Also mind the fact that you are using zones and performing changes on them. Be mindful of which file is which; where are the zones aggregated and where not?

In [ ]:
# This code block retrieves a network 
network = road_network_from_place("Brussels", buffer_dist_close=40000)
network = relabel_graph(network)

# This code block adds centroids to the network
zoning = gpd.read_file(zoning_path)

zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
x_centroids, y_centroids = transform(Proj(init=zoning.crs),Proj(init='epsg:4326'),x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
network = add_centroids(network, x_centroids,y_centroids,k=1, method='link')
network = relabel_graph(network)
# show_network(network, notebook=True)

4. What we need to export to QGIS:

- All intersection nodes, with attributes: x coordinate, y coordinate and node_id. 
- All non connector links, with attributes: link_id, from_node_id and to_node_id. 

In [ ]:
# Remove centroids 
nr_of_centroids = len(x_centroids)
intersection_nodes = list(network.nodes.data())[nr_of_centroids:]

# Remove connectors
edge_data = [(_, _, data) for _, _, data in network.edges.data()]
sorted_edges = sorted(edge_data, key=lambda t: t[2]["link_id"])
link_types = np.array([np.int8(d.get("link_type", 0)) for (_, _, d) in sorted_edges], dtype=np.int8)
connector_ids = []
for i in range(len(link_types)):
    if link_types[i] == -1 or link_types[i] == 1:
        connector_ids.append(i)
for i in reversed(connector_ids):
    del sorted_edges[i]

print(len(intersection_nodes)) # This print should be equal to the number of nodes that was printed above, under the cell block where we retrieve the network. 
print(len(sorted_edges)) # This print should be equal to the number of links that was printed above, under the cell block where we retrieve the network. 

In [ ]:
node_ids = []
x_coord = []
y_coord = []
for _, data in intersection_nodes:
    node_ids.append(data['node_id'])
    x_coord.append(data['x_coord'])
    y_coord.append(data['y_coord'])
x_coord, y_coord = transform(Proj(init='epsg:4326'),Proj(init=zoning.crs),x_coord,y_coord)

link_ids = []
from_node_ids = []
to_node_ids = []
for _, _, data in sorted_edges:
    link_ids.append(data['link_id'])
    from_node_ids.append(data['from_node_id'])
    to_node_ids.append(data['to_node_id'])

In [ ]:
# Create Point shapefile for the nodes 
nodes = []

# Iterate over the node lists and add each node to the dictionary
for node_id, x, y in zip(node_ids, x_coord, y_coord):
    nodes.append({'node_id': node_id, 'x': x, 'y': y})

points = [Point(node['x'], node['y']) for node in nodes]

nodes_shapefile = gpd.GeoDataFrame(nodes, geometry=points)
nodes_shapefile.crs = {'init': zoning.crs}
nodes_shapefile.to_file(node_shapefile_path)


In [ ]:
# Create Line shapefile for the links 
links = []
lines = []

for link_id, from_id, to_id in zip(link_ids, from_node_ids, to_node_ids):
    links.append({'link_id': link_id, 'from_node_id': from_id, 'to_node_id': to_id})
    from_node = [node for node in nodes if node['node_id'] == from_id][0]
    to_node = [node for node in nodes if node['node_id'] == to_id][0]
    line = LineString([(from_node['x'], from_node['y']), (to_node['x'], to_node['y'])])
    lines.append(line)

links_shapefile = gpd.GeoDataFrame(links, geometry=lines)
links_shapefile.crs = {'init': zoning.crs}
links_shapefile.to_file(link_shapefile_path)

    Congratulations, you have been able to export everything you need to QGIS, needed to find accurate cordon- or zone-based tolling schemes!